In [ ]:
pip install --quiet -U langchain-community langchain-ollama langchain-core tiktoken langchain-nomic "nomic[local]"
pip install scikit-learn "nomic[local]" pdfminer.six

In [ ]:
import os, getpass


def _set_env(env_var):
    os.environ[env_var] = os.getenv(env_var)

_set_env("TAVILY_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("LANGCHAIN_TRACING_V2")
_set_env("LANGCHAIN_ENDPOINT")
_set_env("LANGCHAIN_PROJECT")

os.environ["TOKENIZERS_PARALLELISM"] = "true"

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)


In [ ]:
from langchain_ollama import ChatOllama
local_llm = 'llama3.2:3b-instruct-fp16'

llm = ChatOllama(model=local_llm, temperature=0.0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0.0, format="json")

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{input}"),
        ("user", "{input}"),
    ]
)

Please run both the command locally

ollama pull llama3.2:3b-instruct-fp16
ollama run llama3.2:3b-instruct-fp16

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFMinerLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_nomic.embeddings import NomicEmbeddings

loader = PDFMinerLoader("data/Algomasterio_System_Design_Interview_Handbook.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")

vector_store = SKLearnVectorStore.from_documents(docs, embeddings)

retriever = vector_store.as_retriever(k=3)

query = "What did the president say about Ketanji Brown Jackson"

docs = vector_store.similarity_search(query)
print(docs[0].page_content)


In [ ]:
retriever.invoke("What is the best way to prepare for a system design interview?")

In [ ]:
documents = retriever.invoke("Tell me all the topics this books covers?")

for doc in documents:
    print(doc.page_content)


In [ ]:
retriever.invoke("What are the best practices for designing scalable systems?")

In [ ]:

retriever.invoke("What are the best practices for designing distributed systems?")

In [ ]:
###Router

import json
from langchain_core.messages import HumanMessage, SystemMessage

#Prompt
router_instructions = """ You are an expert at routing a user question to Vectorstore to web search.
The vector store contains a handbook on system design interview.
The web search contains the most recent information on the internet.
You will be given a user query and you need to decide which tool to use.
You can either choose to search the web or search the vector store.
You can only choose one of the options.
If you can answer the user question, return "I can answer that!"
If not, you need to find the most relevant information in the vector store or decide to search the web.
return the json with the following schema:
{
    "choice": "vector_store" or "web_search",
    "reason": "reason for choosing the tool"
    "search_query": "search query to use"
    "search_k": "number of results to return"
}
"""

question = [HumanMessage(content="What is the best way to prepare for a system design interview?")]
test_vector_store = llm_json_mode.invoke([SystemMessage(content=router_instructions)]+ question)
json.loads(test_vector_store.content)